In [ ]:
%cd /Users/rodrigo/Post-Grad/CC400/Repo

In [ ]:
import utils as ut

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch_geometric
from torch_geometric.data import Data
from torch_geometric.utils import dense_to_sparse
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
from sklearn.model_selection import cross_validate
from tqdm import tqdm
from torch_geometric.loader import DataLoader
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from neurocombat_sklearn import CombatModel

torch.set_default_dtype(torch.float64)
np.random.seed(42)

%matplotlib inline
sns.set(rc={'image.cmap': 'coolwarm'})

#from numba import jit,prange

import time
import os

SMALL_SIZE = 8
MEDIUM_SIZE = 14
BIGGER_SIZE = 20

plt.rc('font', size=MEDIUM_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=BIGGER_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=MEDIUM_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

scoring = {'acc': 'accuracy',
           'prec_macro': 'precision_macro',
           'rec_macro': 'recall_macro',
          'f1' : 'f1_macro',
          'roc_auc' : 'roc_auc'}


In [ ]:
df, phenotypic = ut.import_data(fisher=False)

#df = df.join(pd.DataFrame(phenotypic.Age), how='left')
#df = df.join(pd.DataFrame(phenotypic['ADHD Measure']), how='left')
df = df.join(pd.DataFrame(phenotypic['Gender']), how='left')

df['Age'] = phenotypic['Age']

from sklearn.utils import shuffle
df = shuffle(df, random_state=42)

df = df.dropna(axis=0)
df = df.reset_index()
df['Site'] = df['Institution'].astype('category')
df['Site'] = df['Site'].cat.codes

# TEST = df[df.Subject.isin(df['Subject'].unique()[-20:])].reset_index()
# X_TEST = TEST.drop(columns=['Institution', 'Subject', 'Run','Gender', 'Age', 'Site','index'])
# y_TEST = TEST.Gender

# df = df[~df.Subject.isin(df['Subject'].unique()[-20:])].reset_index().drop(columns='index')

#X = df.drop(columns=['Institution', 'Run', 'Age','ADHD Measure', 'Gender', 'Subject'])
y = df.Gender.astype(int)


In [ ]:
df_train = df[~df.Subject.isin(np.random.choice(df['Subject'].unique(),100))]
df_test = df[df.Subject.isin(np.random.choice(df['Subject'].unique(),100))]

Age_train = df_train[['Age']]
Site_train = df_train[['Site']]
X_train = df_train.drop(columns=['Institution', 'Subject', 'Run','Gender', 'Age', 'Site', 'Half'])
y_train = df_train.Gender

Age_test = df_test[['Age']]
Site_test = df_test[['Site']]
X_test = df_test.drop(columns=['Institution', 'Subject', 'Run', 'Gender', 'Age', 'Site', 'Half'])
y_test = df_test.Gender

In [ ]:

# Creating model
model = CombatModel()

# Fitting the model and transforming the training set
X_train = model.fit_transform(X_train.values,
                                         Site_train) #X_train_har

# Harmonize test set using training set fitted parameters
X_test = model.transform(X_test.values,
                                    Site_test) #X_test_har

In [ ]:
img_train = np.zeros((X_train.shape[0],1, 190, 190))
for i in range(X_train.shape[0]):
    img_train[i, :, :] = ut.reconstruct_symmetric_matrix(190,X_train[i,:])
    
img_test = np.zeros((X_test.shape[0], 1, 190, 190))
for i in range(X_test.shape[0]):
    img_test[i, :, :] = ut.reconstruct_symmetric_matrix(190,X_test[i,:])

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch.nn.functional as func

class ImageDataset(Dataset):
    def __init__(self, images, labels, transform=None, target_transform=None):
        self.images = torch.from_numpy(images)
        self.img_labels = torch.from_numpy(labels.values.astype(int))
        self.transform = transform
        self.target_transform = target_transform
        # self.transform = transforms.Compose([
        #     transforms.ToTensor(),
        #     transforms.Normalize((0.5,), (0.5,))
        # ])

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        image = self.images[idx]
        #image = self.transform(image)
        label = self.img_labels[idx]
        return image,label

In [ ]:
img_train = ImageDataset(img_train, y_train)
train_loader = DataLoader(img_train, batch_size=32)

img_test = ImageDataset(img_test, y_test)
test_loader = DataLoader(img_test, batch_size=32)

In [ ]:
len(img_train)

In [ ]:
xs, ys = img_train[0:4]
print(xs.shape)

In [ ]:
# With square kernels and equal stride
m = nn.Conv2d(1, 33, 3, stride=2)
n = nn.ReLU()

In [ ]:
print('Images .shape \n', xs.shape)
print('Images after CNN .shape \n',m(xs).shape)
print('Images after CNN -> ReLU .shape \n', n(m(xs)).shape)

In [ ]:
class NN(nn.Module):
    def __init__(self,dropout=.5):
        super(NN,self).__init__()
        
        self.conv1 = nn.Conv2d(1, 32, kernel_size=(2,2), stride=1)
        self.pool1 = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(32, 2, kernel_size=(2,2), stride=1)
        self.pool2 = nn.MaxPool2d(2,2)
        self.conv3 = nn.Conv2d(16, 8, kernel_size=(4,4), stride=1)
        self.pool3 = nn.MaxPool2d(2,2)
        
        self.lin1 = nn.Linear(4232, 2)
        #self.R = nn.LeakyReLU()
        self.R = nn.Tanh()
        self.p = dropout
        
    def forward(self,x):
        x = self.R(self.conv1(x))
        x = self.R(self.pool1(x))
        x = func.dropout(x, p=self.p, training=self.training)
        x = self.R(self.conv2(x))
        x = self.R(self.pool2(x))
        #x = func.dropout(x, p=self.p, training=self.training)
        #x = self.R(self.conv3(x))
        #x = self.R(self.pool3(x))
        #x = x.view(-1,4*(46**2))
        x = torch.flatten(x,1)
        x = self.lin1(x)
        return x.squeeze()

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [ ]:
f = NN()
count_parameters(f)

In [ ]:
xs.shape

In [ ]:
f(xs).shape

In [ ]:
# for batch, (X, y) in enumerate(train_loader):
#     print(X)

In [ ]:
# Display image and label.
train_features, train_labels = next(iter(train_loader))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")
img = train_features[0].squeeze()
label = train_labels[0]
plt.imshow(img, cmap="gray")
plt.show()
print(f"Label: {label}")


In [ ]:
from torch.autograd import Variable

def train_loop(dataloader, model, optimizer, loop):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.train()
    loss_all = 0
    pred = []
    label = []
    
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    for batch, (X, y) in enumerate(loop):
        
        # Zero your gradients for every batch!
        optimizer.zero_grad()
        
        y = y.to(device)
        # Compute prediction and loss
        output = model(X)
        #print(output.shape)
        loss = func.cross_entropy(output, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        loss_all += loss.item()*len(y)
        
        loop.set_description(f"Epoch [{epoch}/{NUM_EPOCHS}]")
        loop.set_postfix(loss=loss_all/len(img_train))
        
        pred.append(func.softmax(output, dim=1).max(dim=1)[1])
        label.append(y)

        # if batch % 100 == 0:
        #     loss, current = loss.item(), (batch + 1) * len(X)
        #     print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
            #optimizer.step()
    y_pred = torch.cat(pred, dim=0).cpu().detach().numpy()
    y_true = torch.cat(label, dim=0).cpu().detach().numpy()
    tn, fp, fn, tp = confusion_matrix(y_pred, y_true).ravel()
    epoch_acc = (tn + tp) / (tn + tp + fn + fp)
        
    return epoch_acc, loss_all / size


def test_loop(dataloader, model):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    loss_all = 0
    pred = []
    label = []

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            output = model(X)
            loss = func.cross_entropy(output, y)
            correct += (output.argmax(1) == y).type(torch.float).sum().item()
            
            loss_all += loss.item()*len(y)
            pred.append(func.softmax(output, dim=1).max(dim=1)[1])
            label.append(y)

    y_pred = torch.cat(pred, dim=0).cpu().detach().numpy()
    y_true = torch.cat(label, dim=0).cpu().detach().numpy()
    tn, fp, fn, tp = confusion_matrix(y_pred, y_true).ravel()
    epoch_rec = tp / (tp + fn)
    epoch_prec = tp / (tp + fp)
    epoch_f1 = 2*(epoch_rec*epoch_prec)/(epoch_rec + epoch_prec)
    epoch_acc = (tn + tp) / (tn + tp + fn + fp)
    
    # AUC & ROC
    fpr, tpr, _ = roc_curve(y_true, y_pred)
    roc_auc = auc(fpr, tpr)
    
    
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {(loss_all / size):>8f} \n")
    
    return epoch_rec, epoch_prec, epoch_acc, loss_all / size, roc_auc,epoch_f1

In [ ]:
metrics = {"loss_train" : [], "loss_test" : [], "acc_test" : [], "acc_train" : []}

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = NN().to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-4)

L = nn.CrossEntropyLoss()

NUM_EPOCHS = 25
for epoch in range(1,NUM_EPOCHS + 1):
    loop = tqdm(train_loader)
    train_acc, train_loss = train_loop(train_loader, model, optimizer, loop)
    test_rec, test_prec, test_acc, test_loss, roc_auc, test_f1 = test_loop(test_loader, model)
    
    
    
    metrics['loss_train'].append(train_loss)
    metrics['loss_test'].append(test_loss)
    metrics['acc_test'].append(test_acc)
    metrics['acc_train'].append(train_acc)

In [ ]:
y_pred = []
for y_i in test_loader:

    #y_pred.append(model(y).detach().numpy())
    y_pred.append(func.softmax(model(y_i[0]), dim=1).detach().numpy())#.max(dim=1)[1])

In [ ]:
plt.figure(figsize=(12,5))
plt.hist(np.array(y_pred), bins=15)
plt.title("Prob distribution")
plt.show()

In [ ]:
fig,ax = plt.subplots(figsize=(10,6))
ax.plot(metrics['loss_train'], label='train')
ax.plot(metrics['loss_test'], label='Validation')
ax.set_ylabel('Cross entropy loss')
ax.set_xlabel('Epochs')
ax.legend()
#plt.xlim(0,100)
plt.show()

In [ ]:
X_train, y_train, X_test, y_test = ut.cross_val_data(df, folds=5, site=True)

In [ ]:
from sklearn.preprocessing import StandardScaler

eval_scores = {"loss_train" : [], "loss_test" : [], "acc_test" : [], "acc_train" : []}

scores = np.zeros((6,5))

# K-fold Cross Validation model evaluation
fold_no = 1
for k in range(5):
    
#     scaler = StandardScaler()
#     X_train[k] = scaler.fit_transform(X_train[k])

#     X_test[k] = scaler.transform(X_test[k])

    img_train = np.zeros((X_train[k].shape[0],1, 190, 190))
    for i in range(X_train[k].shape[0]):
        img_train[i, :, :] = ut.reconstruct_symmetric_matrix(190,X_train[k][i,:])

    img_test = np.zeros((X_test[k].shape[0], 1, 190, 190))
    for i in range(X_test[k].shape[0]):
        img_test[i, :, :] = ut.reconstruct_symmetric_matrix(190,X_test[k][i,:])
        
    img_train = ImageDataset(img_train, y_train[k])
    train_loader = DataLoader(img_train, batch_size=64)

    img_test = ImageDataset(img_test, y_test[k])
    test_loader = DataLoader(img_test, batch_size=64)
    
    
    
    metrics = {"loss_train" : [], "loss_test" : [], "acc_test" : [], "acc_train" : []}

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = NN().to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-3)


    NUM_EPOCHS = 10
    for epoch in range(1,NUM_EPOCHS + 1):
        loop = tqdm(train_loader)
        train_acc, train_loss = train_loop(train_loader, model, optimizer, loop)
        test_rec, test_prec, test_acc, test_loss, roc_auc, test_f1 = test_loop(test_loader, model)



        eval_scores['loss_train'].append(train_loss)
        eval_scores['loss_test'].append(test_loss)
        eval_scores['acc_test'].append(test_acc)
        eval_scores['acc_train'].append(train_acc)
        
        print('Val Accuracy {} , Val Loss {}'.format(test_acc, test_loss))
        print('Train Accuracy {} , Train Loss {}'.format(train_acc, train_loss))

    
    scores[0][fold_no - 1] = test_acc
    scores[1][fold_no - 1] = test_rec
    scores[2][fold_no - 1] = test_prec
    scores[3][fold_no - 1] = test_loss
    scores[4][fold_no - 1] = roc_auc
    scores[5][fold_no - 1] = test_f1

    print(f'Score for fold {fold_no}: loss of {test_loss}; acc of {test_acc}%; AUC of {roc_auc}%')

    # Increase fold number
    fold_no = fold_no + 1